In [12]:
#importing the Libraies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_predict
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
rf = RandomForestClassifier(random_state=42)


param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,               # Number of random combinations to try
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# Fit the model
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 300]},
                   random_state=42, scoring='accuracy', verbose=1)

In [13]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(random_search.best_estimator_, X, y, cv=5)

In [14]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [15]:
print("Best Parameters:", random_search.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 10, 'bootstrap': True}
The f1_macro: 0.87
The confusion Matrix:
 [[231  26]
 [ 26 117]]
The report:
               precision    recall  f1-score   support

           0       0.90      0.90      0.90       257
           1       0.82      0.82      0.82       143

    accuracy                           0.87       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.87      0.87      0.87       400



In [16]:
# Save Best Model
import pickle
filename = "SVC_CV_best_model.sav"
pickle.dump(random_search.best_estimator_, open(filename, 'wb'))

In [17]:
# CV Results table
results_df = pd.DataFrame(random_search.cv_results_)
display(results_df.head())

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_bootstrap,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.253050,0.011515,0.017237,0.001266,100,10,1,10,True,"{'n_estimators': 100, 'min_samples_split': 10,...",0.943925,0.915888,0.867925,0.909246,0.031381,1
1,0.183935,0.001324,0.016477,0.000479,100,2,2,None,False,"{'n_estimators': 100, 'min_samples_split': 2, ...",0.915888,0.897196,0.867925,0.893670,0.019739,15
2,0.094196,0.000907,0.010092,0.000145,50,5,1,30,False,"{'n_estimators': 50, 'min_samples_split': 5, '...",0.897196,0.897196,0.858491,0.884294,0.018246,18
3,0.514045,0.022005,0.033860,0.005829,200,5,1,10,True,"{'n_estimators': 200, 'min_samples_split': 5, ...",0.934579,0.897196,0.867925,0.899900,0.027279,12
4,0.276650,0.074162,0.033753,0.013849,100,2,1,10,False,"{'n_estimators': 100, 'min_samples_split': 2, ...",0.869159,0.878505,0.858491,0.868718,0.008177,20
